### 1. ENV

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 2. LLM

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

import os

os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY4"]
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=1,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

### 3. Neo4j

In [3]:
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
NEO4J_URI="neo4j+s://2d5e8539.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="xn8iCGEj2vymA3-43-57qlL63CD70SthzTE_Mt8QfG0"
NEO4J_DATABASE="neo4j"
AURA_INSTANCEID="2d5e8539"
AURA_INSTANCENAME="Instance01"


from langchain_neo4j import Neo4jGraph
enhanced_graph = Neo4jGraph(
    url=NEO4J_URI,
    username="neo4j",
    password=NEO4J_PASSWORD,
    driver_config={
        "max_connection_lifetime": 300,  # 5 minutes
        "keep_alive": True,
        "max_connection_pool_size": 50
    },
    enhanced_schema=True)

### 4. Tools

In [4]:
from langraph_neo4j3 import AgentState, run_agent_workflow
from langchain_core.tools import tool

@tool
def query_tool(query):
    """This tool can query data from graph database using english language"""
    state: AgentState = {
            "question": query,
            "next_action": "",
            "cypher_errors": [],
            "database_records": [],
            "steps": [],
            "answer": "",
            "cypher_statement": ""
        }
    result = run_agent_workflow(state,enhanced_graph)
    return result["answer"]
tools=[query_tool]

### 5. State Graph

In [5]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    # loop_count:int
    # answer:str

graph_builder = StateGraph(State)


### 6. Tool Node

In [6]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

tool_node = BasicToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

### 7. Chatbot Node

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """

You are a conversational agent with reasoning and data interpretation capabilities, working with a graph database.

Rules:
1. If the question is unrelated to the graph, reply normally (no tool use).
2. If the question can be answered with a single direct query → call query_tool ONCE with a plain English request. 
   - After receiving the result, return it immediately. Do not query further unless the user explicitly asks.
3. If the question is indirect or requires reasoning:
   - Form a hypothesis.
   - Iteratively call query_tool to test hypotheses.
   - Refine results step by step.
   - Then summarize findings.
4. Never assume the user wants related data unless explicitly stated. Only query what is directly requested.


### Schema
---
{schema}
---

Always:  

- Never generate Cypher.  
- Use plain English instructions with `query_tool`.  
- Use the schema above to understand what entities and relationships are available before deciding whether to query.
- Stop as soon as the user’s explicit question is answered. Do not generate additional queries beyond what the user asked for.
    """
            ),
            (
                "human",
        """User Question:
    {question}
    """
            ),
        ]
    )

llm_with_tools = llm.bind_tools(tools)
chain = prompt | llm_with_tools

def chatbot(state: State):
    # state["loop_count"]=state["loop_count"]+1
    return {"messages":[chain.invoke({
                "question": state['messages'],
                "schema": enhanced_graph.schema,
            })]}

graph_builder.add_node("chatbot", chatbot)


In [8]:
def route_tools(state: State,):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "END" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", END: END},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [9]:
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": user_input}):
        for value in event.values():
            msg = value["messages"][-1]

            if isinstance(msg, AIMessage):
                if msg.tool_calls:  # AI is requesting a tool
                    print("\n🤖 AIMessage (Tool Call Request)")
                    for tool_call in msg.tool_calls:
                        print("Arguments:", json.dumps(tool_call["args"], indent=2))
                else:  # Normal AI response
                    print("\n🤖 AIMessage")
                    print("Content:", msg.content)

            elif isinstance(msg, ToolMessage):
                print("\n🛠️ ToolMessage (Tool Result)")
                print("Tool Name:", msg.name)
                print("Tool Call ID:", msg.tool_call_id)
                print("Content:", msg.content)

            elif isinstance(msg, HumanMessage):
                print("\n👤 HumanMessage")
                print("Content:", msg.content)

            elif isinstance(msg, SystemMessage):
                print("\n⚙️ SystemMessage")
                print("Content:", msg.content)

            else:
                print("\n❓ Unknown Message Type:", type(msg))
                print(msg)

while True:
    try:
        user_input = input("\nUser: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)

    except Exception as e:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("\nUser:", user_input)
        stream_graph_updates(user_input)

        break


🤖 AIMessage (Tool Call Request)
Arguments: {
  "query": "What were the total sales for the year 2004?"
}

🛠️ ToolMessage (Tool Result)
Tool Name: query_tool
Tool Call ID: 93f18b42-91c5-484e-bcf3-f86e208ab0d7
Content: "The total sales for the year 2004 were 4,724,162.60."

🤖 AIMessage (Tool Call Request)
Arguments: {
  "query": "What were the total sales for the year 2005?"
}

🛠️ ToolMessage (Tool Result)
Tool Name: query_tool
Tool Call ID: c3bef01e-07b3-4cd9-9035-9f5f52a24888
Content: "The total sales for the year 2005 were 1,791,486.71."

🤖 AIMessage (Tool Call Request)
Arguments: {
  "query": "What are the counts of orders by status for the year 2004?"
}

🛠️ ToolMessage (Tool Result)
Tool Name: query_tool
Tool Call ID: d9caf3bc-3af5-4856-8d32-812f513987d8
Content: "For the year 2004, the order counts by status are:\n*   **Shipped**: 139\n*   **Cancelled**: 3\n*   **On Hold**: 1\n*   **Resolved**: 1"


KeyboardInterrupt: 